# 1. 트랜스포머 모델의 개괄

### 1.1. 트랜스포머란?


왼쪽이 디코더(Decoder) 레이어, 오른쪽이 인코더(Encoder) 레이어

셀프 어텐션과 멀티 헤드 어텐션을 수행함

# 2. 어텐션 메커니즘(Attention Mechanism)



그래서 트랜스포머 구조를 이해하기 위해 **어텐션 메커니즘**을 이해할 필요가 있음

CNN을 알기위해 Convolution을 알아야 하는것 처럼



### 2.1. 기본 아이디어

디코더에서 출력 예측하는 시점마다 인코더의 전체 입력을 다시 참고함

여기서 전체 입력 중 예측하는 부분과 연관된 부분에 더 집중(Attenrion)해서 보기 때문에 

    어텐션 메커니즘(Attention Mechanism)

## 2.2 구성요소

    아래의 설명은 Self Attention을 기준으로 함
    쿼리(Query), 키(Key), 벨류(Value)

위의 3가지 구성요소는 입력데이터($x$)에 대한 "선형변환" **(벡터와 행렬의 곱) $ = [vector][matrix]$** 으로 생성됨
$$Q(Query) = xW_Q$$
$$K(Key) = xW_K$$
$$V(Value) = xW_V$$

$W_Q,W_K,W_V$는 가중치 행렬이며 모델이 학습하며 업데이트하는 값

$W_Q,W_K,W_V$는 임베딩 차원($d_{model}$)과 헤드의 개수($num\;head$)에 기반해 생성

임베딩 차원과 헤드가 뭔지는 트랜스포머 모델을 설명할 때


### 2.2.1 임베딩
입력 데이터에 대한 고차원의 연속적인 특징 공간
"The cat sat on the mat"를 입력 데이터라 할 때 $d_{model} = 512$ 라고 하자

입력데이터에 대한 임베딩 결과는 $$x.shape = [6,512] = [입력데이터의\;길이,d_{model}]$$


In [20]:
import torch
data = "The cat sat on the mat"
d_model = 512

def embedding(data,d_model):
    #단어가 입력된 상황을 가정하고 토큰화 진행
    tokens = data.split()
    #토큰의 개수, d_model의 임베딩 생성됨
    return torch.rand(len(tokens), d_model)

embedding_matrix  = embedding(data,d_model)
print("임베딩 결과의 크기",embedding_matrix.shape)
print("임베딩 결과",embedding_matrix)

임베딩 결과의 크기 torch.Size([6, 512])
임베딩 결과 tensor([[0.9851, 0.7044, 0.6417,  ..., 0.3498, 0.6813, 0.5172],
        [0.7140, 0.8189, 0.6091,  ..., 0.1643, 0.1062, 0.4383],
        [0.8386, 0.8516, 0.8611,  ..., 0.6039, 0.0629, 0.0511],
        [0.0476, 0.0713, 0.8978,  ..., 0.0592, 0.2063, 0.8426],
        [0.8651, 0.2321, 0.9658,  ..., 0.6976, 0.5677, 0.4458],
        [0.4409, 0.8931, 0.8318,  ..., 0.0290, 0.9022, 0.8148]])


### 2.2.2 멀티-헤드

트렌스포머 모델은 입력 데이터를 여러 헤드에서 동시에 처리함
각 헤드마다 다른 관점에서 분석

입력 데이터에 임베딩에는 사용되지 않음

쿼리,키,벨류의 가중치 행렬 생성시 $shape = [d_{model},{d_{model} \over num\;head}]$으로 생성함

## 2.3 쿼리(Query)

현재 단어를 나타내는 벡터를 의미함

쿼리의 가중치 행렬

$$W_Q = \begin{bmatrix}
q_{11} & \cdots & q_{1(num\;head)}\\
\vdots & \ddots & \vdots \\
q_{(d_{model})1} & \cdots & q_{(d_{model})(num\;head)}\\
\end{bmatrix}$$

쿼리 가중치 행렬의 크기
$$W_Q.shape = [d_{model},{d_{model} \over num\;head}]$$

Q의 크기
$$Q.shape = [입력데이터의\;길이,{d_{model} \over num\;head}]$$



#### - 예시
"The cat sat on the mat"에서 "cat"이라는 단어를 분석한다고 가정할 때,$d_{model} = 512,num\;head=8 $ 이라고 하자

 "cat"에 대응되는 **쿼리 벡터**를 찾고싶다.



입력 데이터의 임베딩($x$)를 $x = \begin{bmatrix}x_{1,1} & \cdots & x_{1,512}\\
\vdots & \ddots & \vdots \\
x_{6,1} & \cdots & x_{6,512}\end{bmatrix}$라 하자,

 임의의 $W_Q = \begin{bmatrix}
q_{1,1} & \cdots & q_{1,64}\\
\vdots & \ddots & \vdots \\
q_{512,1} & \cdots & q_{512,64}\\
\end{bmatrix}$가 있을 때,
 
  선형변환($=xW_Q$) 된 결과
  
$$Q =
\begin{bmatrix}x_{1,1} & \cdots & x_{1,512}\\
\vdots & \ddots & \vdots \\
x_{6,1} & \cdots & x_{6,512}\end{bmatrix}
\begin{bmatrix}
q_{1,1} & \cdots & q_{1,64}\\
\vdots & \ddots & \vdots \\
q_{512,1} & \cdots & q_{512,64}\\
\end{bmatrix} 
=\begin{bmatrix}
(x_{1,1})(q_{1,1}) +\cdots + (x_{1,512})(q_{512,1})& \cdots & (x_{1,1})(q_{1,64}) +\cdots +(x_{1,512}q_{512,64}) \\
\vdots & \ddots & \vdots \\
(x_{6,1})(q_{1,1}) +\cdots + (x_{1,512})(q_{512,1})& \cdots & (x_{6,1})(q_{1,64}) +\cdots + (x_{6,512})(q_{512,64})\\
\end{bmatrix}$$

$$Q.shape = [6,64] = [입력데이터길이,{d_{model} \over num\;head}]$$

$$\begin{bmatrix} (x_{2,1})(q_{1,2}) +\cdots + (x_{2,512})(q_{512,2}) & ,\;\cdots & ,(x_{2,1})(q_{1,64}) +\cdots + (x_{2,512})(q_{512,64})\end{bmatrix}\;이\;"cat"에\;대한\;쿼리 벡터\; 이다.$$

In [8]:
import torch
import torch.nn as nn
# 설정
d_model = 512
num_heads = 8
head_dim = d_model // num_heads

# 랜덤함수로 임의의 임베딩 벡터 및 가중치 행렬 생성
embedding_matrix = embedding(data,d_model)  # 6개 단어에 대한 임베딩
W_q = torch.rand(d_model, head_dim)  # 쿼리 가중치 행렬

print("문장 임베딩시 벡터의 크기",embedding_matrix.shape)
print("쿼리 가중치 행렬의 크기",W_q.shape)
# "cat"에 해당하는 임베딩 벡터 (예제에서는 두 번째 단어로 가정)
cat_embedding = embedding_matrix[1, :]  # "cat"의 임베딩

# "cat"에 대한 쿼리 벡터 계산
cat_query = torch.matmul(cat_embedding, W_q)
print()
print("cat에 대한 쿼리 벡터의 크기:", cat_query.shape)
print("cat에 대한 쿼리 벡터:", cat_query)


문장 임베딩시 벡터의 크기 torch.Size([6, 512])
쿼리 가중치 행렬의 크기 torch.Size([512, 64])

cat에 대한 쿼리 벡터의 크기: torch.Size([64])
cat에 대한 쿼리 벡터: tensor([126.7530, 126.7477, 125.9892, 125.6708, 121.0673, 118.2242, 120.0349,
        126.1817, 121.5843, 127.4763, 125.1348, 117.7800, 125.4988, 127.0261,
        122.9178, 123.5058, 122.0167, 126.1502, 124.4548, 121.0811, 129.9183,
        124.8791, 125.2189, 125.8380, 128.4730, 127.3037, 124.0913, 125.2077,
        117.9322, 130.1100, 126.5133, 119.5291, 123.4935, 133.0323, 115.3479,
        129.5890, 126.6831, 119.4234, 123.9116, 126.4938, 126.4543, 128.1191,
        129.0816, 117.0037, 119.4417, 122.7524, 128.6954, 120.4455, 117.2934,
        122.0824, 129.8965, 126.2648, 124.1676, 127.9894, 125.1102, 120.4204,
        120.0303, 119.3966, 124.3703, 120.8507, 120.1961, 120.5619, 124.1685,
        126.2433])


## 2.4 키(Key) - 벨류(Value)

- 기본개념: 키(Key) - 벨류(Value) 구조는 흔히 아래와 같은 딕셔너리 형에서 사용됨

    키값으로 벨류를 가져올 수 있다.


In [4]:
dict = {"2017" : "Transformer", "2018" : "BERT"}
print(dict["2018"])

BERT


키(Key)가중치 행렬 

$$W_K = \begin{bmatrix}
k_{11} & \cdots & k_{1(num\;head)}\\
\vdots & \ddots & \vdots \\
k_{(d_{model})1} & \cdots & k_{(d_{model})(num\;head)}\\
\end{bmatrix}$$

가중치 행렬의 크기
$$W_K.shape = [d_{model},{d_{model} \over num\;head}]$$

K의 크기
$$K.shape = [입력데이터의\;길이,{d_{model} \over num\;head}]$$

벨류(Value)가중치 행렬 

$$W_V = \begin{bmatrix}
v_{11} & \cdots & v_{1(num\;head)}\\
\vdots & \ddots & \vdots \\
v_{(d_{model})1} & \cdots & v_{(d_{model})(num\;head)}\\
\end{bmatrix}$$

가중치 행렬의 크기
$$W_V.shape = [d_{model},{d_{model} \over num\;head}]$$

V의 크기
$$V.shape = [입력데이터의\;길이,{d_{model} \over num\;head}]$$

#### 쿼리,키,벨류의 가중치 행렬은 모두 같은 크기를 가진다. 

## 2.5 Attention
### 2.5.1 Attention Score 계산
어텐션 스코어는 쿼리와 키의 관계를 나타낸다.

입력 시퀀스(=현재 쿼리)의 대한 전체요소들과의 가중치를 계산하는 것이다.

이 계산 방법에 따라 **다양한 방법**으로 구현 될 수 있다. 그 중 내적(Dot-Product)을 이용한 계산을 알아보자면
$$ Attention(Q,K,V) = softmax({QK^T \over \sqrt{d_k}}) $$
로 정의되며 싱글 헤드 어텐션과 멀티 헤드 어텐션일때 계산이 달라진다.

 - **멀티-헤드 어텐션**
$$ Attention(Q,K,V) = softmax({QK^T \over \sqrt{d_k}}) = softmax({QK^T \over \sqrt{{d_{model} \over num\;head}}})$$

 - 싱글-헤드셀프 어텐션(별로 중요하지 않아보인다. )
 $$ Attention(Q,K,V)= softmax({QK^T \over \sqrt{d_k}}) = softmax({QK^T \over \sqrt{{d_{model} \over 1}}}) = softmax({QK^T \over \sqrt{d_{model}}}) $$



In [31]:
#어텐션 스코어 계산 예제
import torch
import torch.nn.functional as F
import torch.nn as nn
# 설정
d_model = 512
num_heads = 8
head_dim = d_model // num_heads

# 랜덤함수로 임의의 임베딩 벡터 및 가중치 행렬 생성
embedding_matrix = embedding(data,d_model)  # 6개 단어에 대한 임베딩
W_q = torch.rand(d_model, head_dim)  # 쿼리 가중치 행렬

print("문장 임베딩시 벡터의 크기",embedding_matrix.shape)
print("쿼리 가중치 행렬의 크기",W_q.shape)
# "cat"에 해당하는 임베딩 벡터 (예제에서는 두 번째 단어로 가정)
cat_embedding = embedding_matrix[1, :]  # "cat"의 임베딩

# "cat"에 대한 쿼리 벡터 계산
cat_query = torch.matmul(cat_embedding, W_q)
###################위와 동일#########################
# 랜덤함수로 임의의 임베딩 벡터 및 가중치 행렬 생성
W_k = torch.rand(d_model, head_dim)  # 키 가중치 행렬
print("키 가중치 행렬의 크기",W_q.shape)
keys = torch.matmul(embedding_matrix, W_k)

# 어텐션 스코어 계산
attention_scores = torch.matmul(cat_query, keys.T) / torch.sqrt(torch.tensor(d_model // num_heads, dtype=torch.float32))
attention_scores = F.softmax(attention_scores, dim=-1)
print()
print("cat에 대한 어텐션 스코어 크기:", attention_scores.shape)
print("cat에 대한 어텐션 스코어:", attention_scores)

문장 임베딩시 벡터의 크기 torch.Size([6, 512])
쿼리 가중치 행렬의 크기 torch.Size([512, 64])
키 가중치 행렬의 크기 torch.Size([512, 64])

cat에 대한 어텐션 스코어 크기: torch.Size([6])
cat에 대한 어텐션 스코어: tensor([0., 0., 1., 0., 0., 0.])


### 2.5.1 Attention Value 계산

어텐션 스코어를 바탕으로 각 벨류에서의 가중치를 부여해 쿼리벡터 하나에서 다른 요소에 대한 **의미**나 **특성** 그리고 **맥락**을 포함한다.

어텐션 스코어에 가중치가 부여된 벨류를 합산한다.

$$ Weighted\;Value = Attention\;Score\times V  $$

#### 예시
앞서 구한 "cat"에 대한 어텐션 스코어를 바탕으로 "cat"에 대한 어텐션 벨류는? (d_model = 512, num head = 8)

벨류벡터는 앞선 쿼리,키와 같이 데이터 임베딩 벡터와 벨류 가중치 행렬의 곱으로 나타난다 
$$xW_V = \begin{bmatrix}x_{1,1} & \cdots & x_{1,512}\\
\vdots & \ddots & \vdots \\
x_{6,1} & \cdots & x_{6,512}\end{bmatrix}
\begin{bmatrix}
v_{1,1} & \cdots & v_{1,64}\\
\vdots & \ddots & \vdots \\
v_{512,1} & \cdots & v_{512,64}\\
\end{bmatrix} 
=\begin{bmatrix}
(x_{1,1})(v_{1,1}) +\cdots + (x_{1,512})(v_{512,1})& \cdots & (x_{1,1})(v_{1,64}) +\cdots +(x_{1,512}v_{512,64}) \\
\vdots & \ddots & \vdots \\
(x_{6,1})(q_{1,1}) +\cdots + (x_{1,512})(v_{512,1})& \cdots & (x_{6,1})(v_{1,64}) +\cdots + (x_{6,512})(v_{512,64})\\
\end{bmatrix} $$

$$V.shape = [6,64] = [입력데이터길이,{d_{model} \over num\;head}]$$

우리가 원하는건 "cat"에 대한 어텐션 벨류 이므로

$$(임의값 사용)\begin{bmatrix} 0&,0&,0&,0&,1&,0\end{bmatrix}
\begin{bmatrix}
(x_{1,1})(v_{1,1}) +\cdots + (x_{1,512})(v_{512,1})& \cdots & (x_{1,1})(v_{1,64}) +\cdots +(x_{1,512}v_{512,64}) \\
\vdots & \ddots & \vdots \\
(x_{6,1})(q_{1,1}) +\cdots + (x_{1,512})(v_{512,1})& \cdots & (x_{6,1})(v_{1,64}) +\cdots + (x_{6,512})(v_{512,64})\\
\end{bmatrix} \;이\;"cat"에\;대한\;어텐션\; 벨류\; 이다.$$

In [37]:
import torch
import torch.nn.functional as F

# 설정
d_model = 512
num_heads = 8
head_dim = d_model // num_heads

# 임의의 임베딩 및 가중치 행렬 생성
embedding_matrix = torch.rand(6, d_model)  # 6개 단어에 대한 임베딩
W_q = torch.rand(d_model, head_dim)  # 쿼리 가중치 행렬
W_k = torch.rand(d_model, head_dim)  # 키 가중치 행렬
W_v = torch.rand(d_model, head_dim)  # 밸류 가중치 행렬

# "cat"에 해당하는 임베딩 벡터 (예제에서는 두 번째 단어로 가정)
cat_embedding = embedding_matrix[1, :]  # "cat"의 임베딩

# 쿼리, 키, 밸류 벡터 계산
cat_query = torch.matmul(cat_embedding, W_q)
keys = torch.matmul(embedding_matrix, W_k)
values = torch.matmul(embedding_matrix, W_v)

# "cat"에 대한 어텐션 스코어 계산
attention_scores = torch.matmul(cat_query, keys.transpose(-2, -1)) / torch.sqrt(torch.tensor(head_dim, dtype=torch.float32))
attention_scores = F.softmax(attention_scores, dim=-1)

# "cat"에 대한 어텐션 벨류 계산
cat_attention_values = torch.matmul(attention_scores, values)

print("cat에 대한 어텐션 벨류 크기:", cat_attention_values.shape)
print("cat에 대한 어텐션 벨류:", cat_attention_values)


cat에 대한 어텐션 벨류 크기: torch.Size([64])
cat에 대한 어텐션 벨류: tensor([127.7655, 125.9784, 119.8620, 127.0230, 129.0132, 131.1996, 129.9930,
        126.8404, 130.2188, 130.1385, 130.6226, 129.0536, 128.1087, 131.8503,
        131.7267, 130.3544, 129.9814, 125.5734, 126.7550, 130.5771, 125.3661,
        130.1339, 126.5389, 128.8256, 132.1857, 124.3674, 126.7825, 134.0955,
        125.8793, 129.5358, 123.1817, 127.0538, 119.0791, 128.5627, 127.5258,
        130.8436, 131.5439, 129.2166, 127.0407, 133.3909, 122.8754, 130.5598,
        123.9540, 133.4200, 133.7713, 130.0197, 127.8870, 133.4268, 121.3879,
        126.7154, 130.1044, 126.4273, 131.8867, 132.6349, 132.6805, 132.7206,
        132.5424, 136.7787, 130.5218, 123.7178, 129.6644, 132.9768, 125.6332,
        126.9372])


In [35]:
# 전체적인 과정 앞선 예제들은 cat 단어에 대한 예시였음
import torch
import torch.nn.functional as F

# 설정
d_model = 512
num_heads = 8
head_dim = d_model // num_heads

# 임의의 임베딩 및 가중치 행렬 생성
embedding_matrix =  embedding(data,d_model)  # 6개 단어에 대한 임베딩
W_q = torch.rand(d_model, head_dim)  # 쿼리 가중치 행렬
W_k = torch.rand(d_model, head_dim)  # 키 가중치 행렬
W_v = torch.rand(d_model, head_dim)  # 밸류 가중치 행렬

# 쿼리, 키, 밸류 벡터 계산
queries = torch.matmul(embedding_matrix, W_q)
keys = torch.matmul(embedding_matrix, W_k)
values = torch.matmul(embedding_matrix, W_v)

# 어텐션 스코어 계산
attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / torch.sqrt(torch.tensor(head_dim, dtype=torch.float32))
attention_scores = F.softmax(attention_scores, dim=-1)

# 어텐션 벨류 계산
attention_values = torch.matmul(attention_scores, values)

print("어텐션 벨류 크기:", attention_values.shape)
print("어텐션 벨류:", attention_values)

어텐션 벨류 크기: torch.Size([6, 64])
어텐션 벨류: tensor([[130.7679, 131.2600, 128.8020, 137.8293, 131.6392, 131.2069, 128.7494,
         134.9128, 133.2825, 126.7834, 135.1894, 131.9203, 130.7347, 130.6917,
         131.1009, 133.5622, 129.8923, 129.5577, 130.1403, 127.2794, 129.7968,
         132.4955, 126.9975, 133.9855, 128.3091, 127.2353, 134.4391, 135.5217,
         129.3004, 135.8638, 134.1209, 125.3821, 130.9818, 137.0243, 130.1015,
         125.1560, 125.8326, 131.4495, 135.0916, 131.1440, 134.2398, 126.6428,
         131.7151, 125.8072, 131.8732, 128.0460, 131.9175, 131.4975, 131.8590,
         134.6504, 137.2288, 132.2314, 125.7504, 128.6073, 126.9401, 128.0023,
         129.5443, 125.4113, 133.1927, 129.3299, 131.8839, 134.9148, 128.5634,
         132.7564],
        [130.7679, 131.2600, 128.8020, 137.8293, 131.6392, 131.2069, 128.7494,
         134.9128, 133.2825, 126.7834, 135.1894, 131.9203, 130.7347, 130.6917,
         131.1009, 133.5622, 129.8923, 129.5577, 130.1403, 127.2794, 129

# 3. 트랜스포머 모델

앞서 설명한 멀티 헤드 어텐션 개념으로 계산한다.

## 3.1. 임베딩 차원과 멀티 헤드 어텐션

임베딩 하는것은 입력데이터를 고정된 크기로 바꾸는 과정이다.
위의 예시처럼 

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Einsum does matrix multiplication for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just a way to do batch matrix multiplication
        attention = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # attention: (N, heads, query_len, key_len)

        if mask is not None:
            attention = attention.masked_fill(mask == 0, float("-1e20"))

        attention = F.softmax(attention / (self.embed_size ** (1/2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out
    
# 임의의 임베딩 및 마스크 생성
embedding_matrix = torch.rand(6, 512)  # 6개 단어에 대한 임베딩
mask = None  # 필요한 경우 마스크를 사용할 수 있습니다.

# 멀티-헤드 어텐션 인스턴스 생성
multi_head_attn = MultiHeadAttention(embed_size=512, heads=8)

# 멀티-헤드 어텐션 수행
out = multi_head_attn(embedding_matrix, embedding_matrix, embedding_matrix, mask)

print(out)



RuntimeError: shape '[6, 512, 8, 64]' is invalid for input of size 3072